In [ ]:
import pandas as pd
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Carregar dados
base = pd.read_csv('base_para_modelo.csv')
base.head()

In [ ]:
#Separar dados
atributos = base.drop(columns = ['id_usuario', 'flag_retido'])
target = base['flag_retido']

atributos_treino, atributos_validacao, target_treino, target_validacao = train_test_split(
    atributos, target,
    test_size = 0.1,
    stratify = target,
    shuffle = True,
    random_state = 1)

In [ ]:
#Definir função para realizar validação cruzada dos dados
def calcular_score_modelo(modelo, dados_treino, dados_validacao, cv):
    #Separar dados
    x_treino, y_treino = dados_treino
    x_validacao, y_validacao = dados_validacao
    
    #Fazer validação cruzada nos dados de treino
    modelo_cv = cross_validate(estimator = modelo, X = x_treino, y = y_treino,
                               scoring = ['accuracy', 'roc_auc'],
                               cv = cv)
    
    #Aplicar modelo nos dados de validação
    modelo.fit(x_treino, y_treino)
    y_pred = modelo.predict(x_validacao)
    
    #Printar resultados
    print('CV model accuracy:  %.3f +/- %.3f'  %(modelo_cv['test_accuracy'].mean(), 
                                              modelo_cv['test_accuracy'].std()))
    print('CV model roc_auc:  %.3f +/- %.3f'  %(modelo_cv['test_roc_auc'].mean(), 
                                             modelo_cv['test_roc_auc'].std()))
    print('Validation accuracy score: %.3f' %accuracy_score(y_validacao, y_pred))
    print('Validation ROC_AUC score: %.3f' %roc_auc_score(y_validacao, y_pred))
    
    return modelo

# Modelo Base

Vamos começar criando um modelo preditivo simples, através do algoritmo Naive Bayes. O objetivo é ter um parâmetro de comparação para a aplicação de algoritmos de predição mais complexos.

A vantagem de utilizar esse algoritmo agora é a inexistência de hiperparâmetros para ajuste, tornando o processo mais rápido.

In [ ]:
#Calcular precisão do modelo construído com o algoritmo Naive Bayes
naive_bayes = GaussianNB()
modelo_01 = calcular_score_modelo(modelo = naive_bayes,
                                  dados_treino = (atributos_treino, target_treino),
                                  dados_validacao = (atributos_validacao, target_validacao),
                                  cv = 10)

Os resultados do primeiro modelo apontam para um problema desafiador, especialmente quando focamos naqueles para os dados de validação. Considerando que os registros classificados como sucesso representam cerca de 36% do total, ambos acurácia e área sobre a curva ROC indicam que o algoritmo de Naive Bayes conseguiu produzir um modelo apenas marginalmente superior a uma classificação aleatória.

Com isso, passamos para uma modelagem ligeiramente mais sofisticada. Nosso objetivo, no entanto, não será somente de melhorar a precisão da nossa modelagem, mas também obter uma forma de identificar quais atributos são mais e menos importantes para nosso estudo.

# Regressão Logística / Seleção de Atributos

A escolha da regressão logística para esse segundo passo se dá um simples motivo. Ainda que se trate de um algoritmo mais complexo que o Naive Bayes, essa modelagem ainda é bastante simples de ser compreendida. Isso significa, em última instância,  em um alto nível de interpretabilidade dos seus resultados, o que é um fator relevante quando queremos selecionar atributos.

In [ ]:
#Calcular precisão do modelo construído com o algoritmo de Regressão Logística
reg_logistica = LogisticRegression(solver = 'liblinear')
modelo_02 = calcular_score_modelo(modelo = reg_logistica,
                                  dados_treino = (atributos_treino, target_treino),
                                  dados_validacao = (atributos_validacao, target_validacao),
                                  cv = 10)

Os resultados mostram uma clara melhora em relação ao modelo anterior. Um outro ponto de destaque é que o modelo parece apresentar poucos problemas de generalização, uma vez que os scores para os dados de validações estão muito próximos daqueles observados nos dados usados na validação cruzada.

Vamos então, avaliar a importância de cada feature para o modelo.

In [ ]:
#Plotar importância dos atributos no modelo de Regressão Logística
importancia_atributos = pd.DataFrame()
importancia_atributos['atributos'] = atributos_treino.columns
importancia_atributos['valor_coef'] = list(modelo_02.coef_.reshape(-1))
importancia_atributos['valor_coef'] = [abs(valor) for valor in importancia_atributos['valor_coef']]
importancia_atributos = importancia_atributos.sort_values(by = 'valor_coef', ascending = False).tail(10)

fig = go.Figure([go.Bar(x = importancia_atributos['valor_coef'], y = importancia_atributos['atributos'], 
                        orientation = 'h')])
fig.update_layout(title = 'Regressão Logística / Importância dos Atributos',
                  template = 'plotly_white',
                  height = 500)
fig.show()

O gráfico acima nos mostra, de maneira geral, uma ampla distribuição de importâncias entre os atributos. Mais importante do que identificar o valor exato para cada coluna, devemos destacar aqueles campos com baixa ou nenhuma importância para o modelo.

In [ ]:
#Aplicar norma sobre valores de coeficiente dos campos
importancia_atributos['valor_coef'] = [abs(x) for x in importancia_atributos['valor_coef']]
importancia_atributos = importancia_atributos.sort_values(by = 'valor_coef')

In [ ]:
#Plotar distribuição dos valores de importância absolutos
fig = go.Figure([go.Histogram(x = importancia_atributos['valor_coef'])])
fig.update_layout(title = 'Distribuição da Importância Absoluta dos Atributos',
                  template = 'plotly_white')
fig.show()

A distribuição acima aponta para uma quantidade significativa de atributos com coeficiente menor que 0.1. Para efeito de comparação, esse valor representa cerca de metade do valor mediano de importância. 

Eliminar esse grupo de campos por completo seria, portanto, exagerado. O que faremos é identificar os atributos na metade inferior desse agrupamento e eliminá-los.

In [ ]:
#Remover atributos de menor importância na Regressão Logística
remover_atributos = importancia_atributos.sort_values(by = 'valor_coef').head(11)['atributos'].values
atributos_treino.drop(columns = remover_atributos, inplace = True)
atributos_validacao.drop(columns = remover_atributos, inplace = True)
print(remover_atributos)

In [ ]:
#Reavaliar modelo com dados filtrados
reg_logistica = LogisticRegression(solver = 'liblinear')
modelo_02_1 = calcular_score_modelo(modelo = reg_logistica,
                                    dados_treino = (atributos_treino, target_treino),
                                    dados_validacao = (atributos_validacao, target_validacao),
                                    cv = 10)

Como podemos evidenciar, a remoção dos 11 atributos não produziu qualquer efeito notável na precisão do modelo, e nem na sua capacidade de generalização. Com isso, partimos para a última fase da nosa modelagem, na qual aplicaremos o algoritmo mais sofisticado até o momento e para o qual utilizaremos técnias de otimização de hiperparâmetros.

# Random Forest / Otimizando Modelo Através dos Hiperparâmetros

Para o modelo final, vamos utilizar o algoritmo Random Forest. Essa escolha se dá principalmente pela alta capacidade de generalização dos modelos originados, dada a construção da previsão através de uma série de modelos menores construídos a partir de amostras aleatórias do conjunto de dados principal.

In [ ]:
#Calcular precisão do modelo construído com o algoritmo Random Forest (sem otimização de hiperparâmetros)
floresta = RandomForestClassifier()
modelo_02 = calcular_score_modelo(modelo = floresta,
                                  dados_treino = (atributos_treino, target_treino),
                                  dados_validacao = (atributos_validacao, target_validacao),
                                  cv = 10)

Inicialmente, os resultados apresentados pelo modelo gerado a partir do algoritmo Random Forest foram ligeiramente piores que os produzidos pela Regressão Logística. No entanto, como já colocado, vamos avançar no processo de otimização do modelo com o objetivo de garantir uma melhora nas métricas de predição.

In [ ]:
#Definição do espaço de possíveis hiperparâmetros
mapa_hparametros = {
    'n_estimators': [50, 100, 250, 500, 1000],
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 2, 3, 5, 10, None],
    'min_samples_leaf': [1, 2, 10, 25, 50, 100],
    'max_features': ['auto', 'log2', None]}

In [ ]:
#Realizar busca aleatório pela melhor configuração de hiperparâmetros
rsc = RandomizedSearchCV(estimator = floresta,
                         param_distributions = mapa_hparametros,
                         n_iter = 10,
                         scoring = ['accuracy', 'roc_auc'],
                         n_jobs = 3,
                         cv = 5,
                         refit = 'accuracy',
                         verbose = 1)

busca_hparametros = rsc.fit(atributos_treino, target_treino)
print(busca_hparametros.best_score_)
floresta_v2 = busca_hparametros.best_estimator_

In [ ]:
#Validar resultados para o melhor estimador encontrado pela busca dos hiperparâmetros
modelo_03 = calcular_score_modelo(modelo = floresta_v2,
                                  dados_treino = (atributos_treino, target_treino),
                                  dados_validacao = (atributos_validacao, target_validacao),
                                  cv = 10)

Como podemos ver, a otimização através dos hiperparâmetros permitiu que encontrássemos um modelo ainda mais preciso que os melhores resultados obtidos anteriormente. Utilizaremos esse modelo de agora em diante para intepretar os resultados da predição e entender como estes podem ser utilizados.

No entanto, não vamos utilizar aqui os valores de predição binários, mas o um valor contínuo entre 0 e 1, que representa a probabilidade de que cada registro se adeque na condição de sucesso da nossa análise. Adotamos essa estratégia por nos dar capacidade de utilizar as previsões de maneira mais sofisticada, criando segmentações de clientes de acordo com probabilidade de retenção.

In [ ]:
#Calcular as previsões para os dados de validação
previsao = modelo_03.predict_proba(atributos_validacao)
previsao = [x[1] for x in previsao]

In [ ]:
#Plotar distribuição dos valores de previsão
fig = go.Figure([go.Histogram(x = previsao, nbinsx = 30)])
fig.update_layout(title = 'Distribuição da Variável Resposta do Modelo de Previsão de Retenção', 
                  template = 'plotly_white')
fig.show()

Analisando a distribuição dos valores previstos para a variável objetivo, temos que uma porção significativa dos casos se posicionam numa região mais central, na qual uma simples previsão binária está mais propensa a errar. Isso reforça nossa escolha pela estratégia adotada.